In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [2]:
df_P = pd.io.parsers.read_csv(filepath_or_buffer = 'data/D0_filt_10mino3.csv',index_col = 0)
print df_P[:5]

                       O3_ppb    UnixTime       e2v03       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                         
7/10/14 20:25              41  1405023930  114.714286  45.800000  11.400000   
7/10/14 20:26              39  1405023991  120.428571  45.814286  11.314286   
7/10/14 20:27              44  1405024053  119.142857  45.857143  11.300000   
7/10/14 20:28              47  1405024110  117.500000  45.900000  11.300000   
7/10/14 20:29              44  1405024167  120.285714  45.900000  11.457143   

                       Zenith Angle [degrees]  days from start  ref_o3_smooth  \
DATE (MM/DD/YYYY)_MST                                                           
7/10/14 20:25                        99.48960                0             41   
7/10/14 20:26                        99.63789                0             39   
7/10/14 20:27                        99.78580                0             44   
7/10/14 20:28                        99.9

In [3]:
df_P.drop(df_P.columns[[7,8]], axis=1, inplace=True)
print df_P[:5]

                       O3_ppb    UnixTime       e2v03       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                         
7/10/14 20:25              41  1405023930  114.714286  45.800000  11.400000   
7/10/14 20:26              39  1405023991  120.428571  45.814286  11.314286   
7/10/14 20:27              44  1405024053  119.142857  45.857143  11.300000   
7/10/14 20:28              47  1405024110  117.500000  45.900000  11.300000   
7/10/14 20:29              44  1405024167  120.285714  45.900000  11.457143   

                       Zenith Angle [degrees]  days from start  
DATE (MM/DD/YYYY)_MST                                           
7/10/14 20:25                        99.48960                0  
7/10/14 20:26                        99.63789                0  
7/10/14 20:27                        99.78580                0  
7/10/14 20:28                        99.93332                0  
7/10/14 20:29                       100.08045           

####Define the slope function.

In [36]:
def find_lag_slope(df_P, int_min, data_col):
    df_P['diff'] = df_P[data_col].diff()
    
    slope = []
    for i in range(int_min,len(df_P['diff'])):
        top = i-int_min
        slope.append(df_P['diff'][top:i].mean())
    
    for i in range(0,int_min):
        slope.insert(i, 'NaN')

    return slope

In [37]:
def find_lead_slope(df_P, int_min, data_col):
    
    slope = []
    for i in range(0,len(df_P[data_col])-int_min):
        top = i + int_min
        slope.append(df_P['diff'][i:top].mean())
    
    for i in range(len(df_P[data_col])-int_min, len(df_P[data_col])):
        slope.insert(i, 'NaN')

    return slope

####Define functions that create features that represent the area under the temp and humidity curves.

In [38]:
def find_lag_integral(df_P, int_min, data_col):
    area_curve = []

    for i in range(int_min,len(df_P[data_col])):
        top = i - int_min
        area_curve.append(np.trapz(df_P[data_col][top:i]))
    
    for i in range(0,int_min):
        area_curve.insert(i, 'NaN')
    
    return area_curve

In [45]:
def find_lead_integral(df_P, int_min, data_col):
    area_curve = []

    for i in range(0,len(df_P[data_col]-int_min)):
        top = i + int_min
        area_curve.append(np.trapz(df_P[data_col][i:top]))
    
    for i in range(len(df_P[data_col]-int_min), len(df_P[data_col])):
        area_curve.insert(i, 'NaN')
    
    return area_curve

In [ ]:
def sliding_integral(df_P, int_min, data_col):
    area_curve = []

    for i in range(0,len(df_P[data_col]-int_min)):
        top = i + int_min
        area_curve.append(np.trapz(df_P[data_col][i:top]))
    
    for i in range(len(df_P[data_col]-int_min), len(df_P[data_col])):
        area_curve.insert(i, 'NaN')
    
    return area_curve

###Call the slope functions and add them to the dataframe.

In [40]:
def make_func_caller_find_lag_integral(df_P, min_time, max_time, interval, column):
    i = min_time
    while i <= max_time:    
        df_P[str(column) + 'int' + 'lag' + str(i)] = find_lag_integral(df_P, i, column)      
        i += interval
    return df_P  

def make_func_caller_find_lead_integral(df_P, min_time, max_time, interval, column):
    i = min_time
    while i <= max_time:    
        df_P[str(column) + 'int' + 'lag' + str(i)] = find_lead_integral(df_P, i, column)      
        i += interval
    return df_P 

def make_func_caller_find_sliding_integral(df_P, min_time, max_time, interval, column):
    i = min_time
    while i <= max_time:    
        df_P[str(column) + 'int' + 'lag' + str(i)] = find_lead_integral(df_P, i, column)      
        i += interval
    return df_P 

def make_func_caller_find_lag_slope(df_P, min_time, max_time, interval, column):
    i = min_time
    while i <= max_time:    
        df_P[str(column) + 'int' + 'lag' + str(i)] = find_lag_slope(df_P, i, column)      
        i += interval
    return df_P 

def make_func_caller_find_lead_slope(df_P, min_time, max_time, interval, column):
    i = min_time
    while i <= max_time:    
        df_P[str(column) + 'int' + 'lag' + str(i)] = find_lead_slope(df_P, i, column)      
        i += interval
    return df_P 

In [47]:
#call the lag int function for o3, temp, and rh
make_func_caller_find_lag_integral(df_P, 1, 115, 1, 'e2v03')
make_func_caller_find_lag_integral(df_P, 1, 115, 1, 'Temp')
make_func_caller_find_lag_integral(df_P, 1, 115, 1, 'Rh')

#call the lead int for o3, temp, and rh
make_func_caller_find_lead_integral(df_P, 1, 115, 1, 'e2v03')
make_func_caller_find_lead_integral(df_P, 1, 115, 1, 'Temp')
make_func_caller_find_lead_integral(df_P, 1, 115, 1, 'Rh')

#call the lag slope for o3, temp, and rh
make_func_caller_find_lag_slope(df_P, 1, 115, 1, 'e2v03')
make_func_caller_find_lag_slope(df_P, 1, 115, 1, 'Temp')
make_func_caller_find_lag_slope(df_P, 1, 115, 1, 'Rh')

#call the lead slope for o3, temp, and rh
make_func_caller_find_lead_slope(df_P, 1, 115, 1, 'e2v03')
make_func_caller_find_lead_slope(df_P, 1, 115, 1, 'Temp')
make_func_caller_find_lead_slope(df_P, 1, 115, 1, 'Rh')

df_P[:5]

,O3_ppb,UnixTime,e2v03,Temp,Rh,Zenith Angle [degrees],days from start,e2v03intlag5,e2v03intlag10,e2v03intlag15,...,Rhintlag103,Rhintlag104,Rhintlag106,Rhintlag107,Rhintlag108,Rhintlag109,Rhintlag111,Rhintlag112,Rhintlag113,Rhintlag114
DATE (MM/DD/YYYY)_MST,,,,,,,,,,,,,,,,,,,,,
7/10/14 20:25,41,1405023930,114.714286,45.800000,11.400000,99.48960,0,0.01428571,-0.05740741,-0.02857143,...,0,0.0008321776,0.002380952,0.001212938,0.000400534,0,0.0002597403,0,0,0
7/10/14 20:26,39,1405023991,120.428571,45.814286,11.314286,99.63789,0,-0.02571429,-0.05857143,-0.01444444,...,0.0008321776,0,0.001212938,0.000400534,0,0,0,0,0,0
7/10/14 20:27,44,1405024053,119.142857,45.857143,11.300000,99.78580,0,-0.04,-0.04285714,0.0009523813,...,0.0008321776,0.003228022,0.001212938,0.0008010681,0.0007936508,0.001048493,0.0007722008,0.0007653062,0.0007585335,0.001503759
7/10/14 20:28,47,1405024110,117.500000,45.900000,11.300000,99.93332,0,-0.05142857,-0.03857143,0.008571429,...,0.003398058,0.002197802,0.0009433962,0.0009345794,0.001190476,0.0009174312,0.0009009009,0.0008928571,0.001643489,0.002005013
7/10/14 20:29,44,1405024167,120.285714,45.900000,11.457143,100.08045,0,-0.06,-0.03142857,0.0152381,...,0.00221914,0.001373626,0.0009433962,0.001201602,0.0009259259,0.0009174312,0.0009009009,0.001658163,0.002022756,0.002005013


In [48]:
#Delete the first and last 115 rows of the dataframe to remove all NaNs.
df_P = df_P.ix[115:len(df_P['e2v03'])-115]
print df_P.ix[len(df_P['e2v03'])-90:len(df_P['e2v03'])]

                       O3_ppb    UnixTime      e2v03       Temp        Rh  \
DATE (MM/DD/YYYY)_MST                                                       
9/3/14 20:35               25  1409776526  92.857143  47.300000  8.500000   
9/3/14 20:36               34  1409776588  94.000000  47.300000  8.528571   
9/3/14 20:37               33  1409776650  96.000000  47.300000  8.571429   
9/3/14 20:38               28  1409776712  99.714286  47.300000  8.514286   
9/3/14 20:39               36  1409776769  98.500000  47.300000  8.600000   
9/3/14 20:40               46  1409776827  98.285714  47.300000  8.528571   
9/3/14 20:41               46  1409776889  95.142857  47.257143  8.500000   
9/3/14 20:42               38  1409776951  90.571429  47.214286  8.400000   
9/3/14 20:43               41  1409777013  88.285714  47.171429  8.385714   
9/3/14 20:44               32  1409777070  89.833333  47.100000  8.400000   
9/3/14 20:45               48  1409777128  88.571429  47.028571  8.414286   

In [69]:
print df_P.ix[len(df_P['e2v03'])-5:len(df_P['e2v03'])]

                       O3_ppb    UnixTime      e2v03       Temp        Rh  \
DATE (MM/DD/YYYY)_MST                                                       
9/3/14 22:25               43  1409783131  82.714286  43.342857  9.085714   
9/3/14 22:26               26  1409783189  84.000000  43.283333  9.233333   
9/3/14 22:27               36  1409783246  84.571429  43.200000  9.371429   
9/3/14 22:28               41  1409783308  88.000000  43.114286  9.457143   
9/3/14 22:29               40  1409783370  90.142857  43.057143  9.442857   

                       Zenith Angle [degrees]  days from start o3_lead_30  \
DATE (MM/DD/YYYY)_MST                                                       
9/3/14 22:25                        128.19138               55   2562.155   
9/3/14 22:26                        128.28881               55   2565.429   
9/3/14 22:27                        128.38539               55   2565.786   
9/3/14 22:28                        128.48113               55   2565.214  

In [55]:
#df_P.drop(df_P.columns[[9]], axis=1, inplace=True)
#print df_P[:5]

In [49]:
#ozone terms
df_P['O3_sqrt'] = np.sqrt(df_P['e2v03'].astype(float))
df_P['O3_sq'] = df_P['e2v03']**2
df_P['O3_cu'] = df_P['e2v03']**3
df_P['ln_O3'] = np.log(df_P['e2v03'])


#temp terms
df_P['temp_sqrt'] = np.sqrt(df_P['Temp'].astype(float))
df_P['temp_sq'] = df_P['Temp']**2
df_P['temp_cu'] = df_P['Temp']**3
df_P['ln_temp'] = np.log(df_P['Temp'])


#rh terms
df_P['rh_sqrt'] = np.sqrt(df_P['Rh'].astype(float))
df_P['rh_sq'] = df_P['Rh']**2
df_P['rh_cu'] = df_P['Rh']**3
df_P['ln_rh'] = np.log(df_P['Rh'])


#ozone/temp interactions
df_P['03_mult_temp'] = df_P['e2v03']*df_P['Temp']
df_P['03_mult_temp_sq'] = (df_P['e2v03']*df_P['Temp'])**2
df_P['03_mult_temp_cu'] = (df_P['e2v03']*df_P['Temp'])**3
df_P['ln_03_mult_temp'] = np.log(df_P['03_mult_temp'])


#ozone/rh interactions
df_P['03_mult_rh'] = df_P['e2v03']*df_P['Rh']
df_P['03_mult_rh_sq'] = (df_P['e2v03']*df_P['Rh'])**2
df_P['03_mult_rh_cu'] = (df_P['e2v03']*df_P['Rh'])**3
df_P['ln_03_mult_rh'] = np.log(df_P['03_mult_rh'])

#temp/rh interactions
df_P['temp_mult_rh'] = df_P['Temp']*df_P['Rh']
df_P['temp_mult_rh_sq'] = (df_P['Temp']*df_P['Rh'])**2
df_P['temp_mult_rh_cu'] = (df_P['Temp']*df_P['Rh'])**3
df_P['ln_temp_mult_rh'] = np.log(df_P['temp_mult_rh'])


#ozone/rh/temp interactions
df_P['03_mult_rh_&_temp'] = df_P['e2v03']*df_P['Rh']*df_P['Temp']
df_P['03_mult_rh_&_temp_sq'] = (df_P['e2v03']*df_P['Rh']*df_P['Temp'])**2
df_P['03_mult_rh_&_temp_cu'] = (df_P['e2v03']*df_P['Rh']*df_P['Temp'])**3
df_P['ln_03_mult_rh_&_temp_cu'] = np.log(df_P['03_mult_rh_&_temp_cu'])

print df_P[:5]

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipython-3.1.0-py2.7.egg/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipython-3.1.0-py2.7.egg/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipython-3.1.0-py2.7.egg/IPytho

                       O3_ppb    UnixTime      e2v03       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                        
7/10/14 22:20              50  1405030829  85.285714  43.800000  11.485714   
7/10/14 22:21              55  1405030891  92.714286  43.800000  11.528571   
7/10/14 22:22              57  1405030952  90.714286  43.800000  11.528571   
7/10/14 22:23              57  1405031009  89.000000  43.800000  11.550000   
7/10/14 22:24              58  1405031066  88.571429  43.857143  11.542857   

                       Zenith Angle [degrees]  days from start e2v03intlag5  \
DATE (MM/DD/YYYY)_MST                                                         
7/10/14 22:20                       113.37064                0   0.02857143   
7/10/14 22:21                       113.45708                0  0.005714284   
7/10/14 22:22                       113.54281                0 -0.008571428   
7/10/14 22:23                       113.62784             

In [50]:
df_P.to_csv(path_or_buf = 'data/D0_raw_features.csv')